In [1]:
import numpy as np
import pandas as pd
from glob import glob
import json
from tqdm.notebook import tqdm
from pymatgen.core import Lattice, Structure, Molecule
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from collections import Counter
from scipy import stats
from pymatgen.core.periodic_table import ElementBase, Element
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN

C:\Users\atifh\anaconda3\lib\site-packages\pymatgen\core\__init__.py:49: UserWarning: Error loading .pmgrc.yaml: [Errno 2] No such file or directory: 'C:\\Users\\atifh\\.pmgrc.yaml'. You may need to reconfigure your yaml file.
  warnings.warn(f"Error loading .pmgrc.yaml: {ex}. You may need to reconfigure your yaml file.")


## Load the target variable as a dictionary

In [2]:
# Load the train target variables
df = pd.read_csv("dichalcogenides_public/targets.csv")
Y_dict = dict(zip(df["_id"].values, df["band_gap"].values))

# Load the sample submission file
# We will use the order of the ids to generate our own submission file
df = pd.read_csv("sample_submission.csv")
Y_test_ids = df["id"].values

# Define all functions here
## 1. Performance metric
## 2. Generate the vector representation of each structure as a dict

In [3]:
# Performance Metric
def metric(true, pred, e=0.02):
    loss = np.abs(true-pred)
    mask = [1 if i<e else 0 for i in loss]
    return np.average(mask)

def flatten(x):
    x = x[0]
    x_dash = list()
    for i in range(len(x)):
        for j in range(len(x[i])):
            x_dash.append(x[i][j])
    return x_dash

# Creates a single vector representation of each structure
def gen_data(files):
    # USELESS FEATURES
    # ================
    # "lattice" all keys under this key have same values for all samples
    # "Charge" is none for all structures
    # structure.formula has duplicates with different band energy as output hence not useful
    # structure.volume is same for all samples
    # structure.frac_coords is same for all samples ???
    # structure.DISTANCE_TOLERANCE is same for all samples
    # structure.lattice.get_miller_index_from_coords(structure.cart_coords) significantly reduced performance
    # structure.lattice.norm(structure.cart_coords) significantly reduced performance (either average or l2 norm)


    # USEFUL FEATURES
    # ================
    # 1. The minimum number of elements (molecules) in a structure is 189, maximum is 192 and average is 190.51
    #    We can directly use these values
    # 2. Structure Density (structure.density.real)
    # 3. The count of each species in every structure is a vector feature
    # 4. structure.distance matrix has shape of (num_molecules x num_molecules). Contains distance between all molecules
    #    Use maximum, minimum and average distances per row or overall from this matrix?
    # 5. structure.atomic_numbers has one value for each molecule
    # 6. structure.cart_coords is 3d position of each molecule

    all_species = ['Mo', 'S', 'Se', 'W']
    X_dict = dict()
    for idx, file in tqdm(enumerate(files), total=len(files)):
        # Convert raw data into json format
        data = json.load(open(file, 'r'))
        # Convert json format to pymatgen format
        structure = Structure.from_dict(data)
        # Convert structure to dataframe for extraction of some features
        df = structure.as_dataframe()
        # Distance matrix
        dm = structure.distance_matrix
        # Get all species in current structure
        curr_species = ["".join([j for j in i if not j.isdigit()]) for i in structure.formula.split()]
        # Get count of all species in current structure
        curr_species_count = [int("".join([j for j in i if j.isdigit()])) for i in structure.formula.split()]
        # Create a count vector representation of species (extremely small improvement in final score)
        count_vec = np.zeros(len(all_species))
        for idx, i in enumerate(curr_species):
            count_vec[all_species.index(i)] = curr_species_count[idx]
        # Get the atomic numbers (extremely small improvement in final score)
        atm_num = structure.atomic_numbers[:189]
        bm = [Element(i).bulk_modulus for i in curr_species]
        ym = [Element(i).youngs_modulus for i in curr_species]
        dos = [Element(i).density_of_solid for i in curr_species]
        clte = [Element(i).coefficient_of_linear_thermal_expansion for i in curr_species]
        mp = [Element(i).melting_point for i in curr_species]
        X_dict[file[file.rindex("\\")+1:file.index(".")]] = [structure.density.real,
                                                             np.average(dm),
                                                             len(structure.atomic_numbers),
                                                             sum([i*j for i, j in zip(bm, curr_species_count)]),
                                                             sum([i*j for i, j in zip(ym, curr_species_count) if i is not None]),
                                                             sum([i*j for i, j in zip(dos, curr_species_count) if i is not None]),
                                                             sum([i*j for i, j in zip(clte, curr_species_count) if i is not None]),
                                                             sum([i*j for i, j in zip(mp, curr_species_count) if i is not None]),
                                                             np.average(np.sum(dm, axis=0)),
                                                            ] + list(atm_num) + list(count_vec)
    return X_dict


## Pair the generated features with corresponding target

In [5]:
X_train_dict = gen_data(glob("dichalcogenides_public\\structures\\*.json"))
X_test_dict = gen_data(glob("dichalcogenides_private\\structures\\*.json"))

X_train, Y = list(), list()
for lattice_id, feat in X_train_dict.items():
    X_train.append(feat)
    Y.append(Y_dict[lattice_id])
X_train, Y = np.asarray(X_train), np.asarray(Y)
print("Train Data Shape:", X_train.shape)

X_test = list()
for id_ in Y_test_ids:
    X_test.append(X_test_dict[id_])
X_test = np.asarray(X_test)
print("Test Data Shape:", X_test.shape)

  0%|          | 0/2966 [00:00<?, ?it/s]

  0%|          | 0/2967 [00:00<?, ?it/s]

Train Data Shape: (2966, 202)
Test Data Shape: (2967, 202)


## Create Cross Validation

In [6]:
num_splits = 5
kfold = KFold(n_splits=num_splits, random_state=27, shuffle=True)
scores1, scores2, scores3 = list(), list(), list()
for index, (train, test) in enumerate(kfold.split(X_train)):
    x_train, x_test = X_train[train], X_train[test]
    y_train, y_test = Y[train], Y[test]
    
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    inds = np.where(y_train<0.6)
    y_classes = np.zeros(len(y_train))
    y_classes[inds] = 1
    x_train_e = np.concatenate((x_train, np.reshape(y_train, (-1, 1))), axis=1)
    
    minority = len(inds[0])
    majority = len(y_train)-minority
    
    # Perform oversampling using SMOTE 
    sm = SMOTE(random_state=42, sampling_strategy={0:majority, 1:minority*2})
    x_train_e, _ = sm.fit_resample(x_train_e, y_classes)
    x_train, y_train = x_train_e[:,:-1], x_train_e[:,-1]
    
    model = ExtraTreesRegressor(random_state=27)
    model.fit(x_train, y_train)
    preds1 = model.predict(x_test)
    scores1.append(metric(y_test, preds1))
    
    model = RandomForestRegressor(random_state=27, max_depth=9, criterion="absolute_error", n_jobs=-1)
    model.fit(x_train, y_train)
    preds2 = model.predict(x_test)
    scores2.append(metric(y_test, preds2))
    
    scores3.append(metric(y_test, 0.1*preds1+0.9*preds2))
    print("Fold", index+1, "\n==========")
    print("\tXGBoost: ", scores1[-1])
    print("\tRandomForest: ", scores2[-1])
    print("\tEnsemble: ", scores3[-1])
    
print("\n\nAverage Score XGBoost:", sum(scores1)/len(scores1))
print("Average Score RandomForest:", sum(scores2)/len(scores2))
print("Average Score Ensemble:", sum(scores3)/len(scores3))

# 0.8081540531793483
# XGBoost: 0.8064
# RandomForest: 0.8155762231647561

C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2328) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1208)
  warnings.warn(


Fold 1 
	XGBoost:  0.8047138047138047
	RandomForest:  0.8164983164983165
	Ensemble:  0.8164983164983165


C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2334) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1206)
  warnings.warn(


Fold 2 
	XGBoost:  0.7740303541315345
	RandomForest:  0.8178752107925801
	Ensemble:  0.8128161888701517


C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2336) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1205)
  warnings.warn(


Fold 3 
	XGBoost:  0.7858347386172007
	RandomForest:  0.8026981450252951
	Ensemble:  0.8026981450252951


C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2314) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1216)
  warnings.warn(


Fold 4 
	XGBoost:  0.7875210792580101
	RandomForest:  0.8145025295109612
	Ensemble:  0.8111298482293423


C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2352) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1197)
  warnings.warn(


Fold 5 
	XGBoost:  0.806070826306914
	RandomForest:  0.8263069139966274
	Ensemble:  0.821247892074199


Average Score XGBoost: 0.7916341606054927
Average Score RandomForest: 0.8155762231647561
Average Score Ensemble: 0.812878078139461


## Finally predict and save the predictions

In [7]:
inds = np.where(Y<0.6)
Y_classes = np.zeros(len(Y))
Y_classes[inds] = 1
X_train_e = np.concatenate((X_train, np.reshape(Y, (-1, 1))), axis=1)

minority = len(inds[0])
majority = len(Y)-minority

sm = SMOTE(random_state=42, sampling_strategy={0:majority, 1:minority*2})
X_train_e, _ = sm.fit_resample(X_train_e, Y_classes)
X_train, Y = X_train_e[:,:-1], X_train_e[:,-1]

model = RandomForestRegressor(random_state=27, max_depth=9, n_estimators=100, criterion="absolute_error", n_jobs=-1)
model.fit(X_train, Y)
preds = model.predict(X_test)

fp = open("final_submission.csv", "w")
fp.write("id,predictions\n")
for id_, pred in zip(Y_test_ids, preds):
    fp.write(id_+","+str(pred)+"\n")
fp.close()

# Average Score: 0.8064682803300005 (XGBoost)
# LB Score: 0.81873 (no OOF)
# LB Score: 0.81671 (with OOF)

# Average Score: 0.8101805009056273 (RandomForest)
# LB Score: 0.82682

# Average Score: 0.8081540531793483 (Ensemble)
# LB Score: 0.82278

C:\Users\atifh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:299: UserWarning: After over-sampling, the number of samples (2916) in class 1 will be larger than the number of samples in the majority class (class #0.0 -> 1508)
  warnings.warn(
